In [90]:
d_names = sorted(["1138-bus", "USpowerGrid", "qh882", "dwt-1005", "poli"])
target_dataset = d_names[0]

# 2, 3はいい感じ

In [91]:
import pandas as pd
from ex_utils.share import ex_path
import json
from pprint import pprint


types = ["empirical", "multi", "single"]

score_from_a = {
    "Aが好ましい": 2,
    "Aが少し好ましい": 1,
    "変わらない": 0,
    "Bが少し好ましい": -1,
    "Bが好ましい": -2,
}

df = pd.read_csv(
    ex_path.joinpath(
        "data/thesis2023-fuga ノードリンク図の評価実験（回答） - フォームの回答 1 (3).csv"
    )
)
with open(ex_path.joinpath("data/ex2/pictures.json")) as f:
    pictures_data = json.load(f)

remove = ["タイムスタンプ", "氏名", "ノードリンク図の評価実験を通してフィードバックがあれば記入してください。"]
df = df.drop(columns=remove)
df = df.rename(
    columns=dict(
        zip(
            df.columns,
            [c.replace(" [どちらが好ましいですか？]", "") for c in df.columns],
        )
    )
)
results = df

n = len(results)
t = len(types)

In [92]:
ans_ab = dict(zip(results.index, [{} for _ in range(len(results.index))]))

for i, row in results.iterrows():
    d = row.to_dict()
    results_map = dict(
        zip(
            types,
            [
                dict(zip(types, [0, 0, 0])),
                dict(zip(types, [0, 0, 0])),
                dict(zip(types, [0, 0, 0])),
            ],
        )
    )

    for uuid in d:
        ans = d[uuid]
        target_data = pictures_data[uuid]
        a = target_data["A"]
        b = target_data["B"]
        dataset = target_data["dataset"]

        score_a = score_from_a[ans]
        score_b = -score_from_a[ans]
        # if dataset == '1138-bus' and (a == "empirical" or b == "empirical"):
        #     print(ans, target_data)

        if dataset == target_dataset:
            results_map[a][b] += score_a
            results_map[b][a] += score_b
    ans_ab[i] = results_map

# n_seed = 3
# n_compare = 2
# for key in ans_ab:
#     for a in types:
#         for b in types:
#             ans_ab[key][a][b] /= n_seed * n_compare

ans_all = dict(
    zip(
        types,
        [
            dict(zip(types, [0, 0, 0])),
            dict(zip(types, [0, 0, 0])),
            dict(zip(types, [0, 0, 0])),
        ],
    )
)
for key in ans_ab:
    for a in types:
        for b in types:
            ans_all[a][b] += ans_ab[key][a][b]

In [98]:
ans_all

{'empirical': {'empirical': 0, 'multi': -13, 'single': -7},
 'multi': {'empirical': 13, 'multi': 0, 'single': 28},
 'single': {'empirical': 7, 'multi': -28, 'single': 0}}

In [94]:
x_it = {}
x_it2 = {}
for key in ans_ab:
    x_it[key] = {}
    x_it2[key] = {}
    for type in types:
        x_it[key][type] = 0
        for b in types:
            x_it[key][type] += ans_ab[key][type][b]
        x_it2[key][type] = x_it[key][type] ** 2

x_i = {}
x_i2 = {}
for type in types:
    x_i[type] = 0
    for key in ans_ab:
        x_i[type] += x_it[key][type]
    x_i2[type] = x_i[type] ** 2

In [106]:
a_i = {}
for type in types:
    a_i[type] = x_i[type] / (t * n)

a_it = {}
for key in ans_ab:
    a_it[key] = {}
    for type in types:
        a_it[key][type] = x_it[key][type] / t - a_i[type]


r_it = {}
for type in types:
    r_it[type] = {}
    for b in types:
        r_it[type][b] = (ans_all[type][b] / n) - (a_i[type] - a_i[b])

In [109]:
x_i2_sum = sum([x_i2[type] for type in types])

In [112]:
x_it2

{0: {'empirical': 121, 'multi': 4, 'single': 81},
 1: {'empirical': 49, 'multi': 64, 'single': 1},
 2: {'empirical': 9, 'multi': 64, 'single': 25},
 3: {'empirical': 49, 'multi': 16, 'single': 9},
 4: {'empirical': 64, 'multi': 36, 'single': 4},
 5: {'empirical': 25, 'multi': 81, 'single': 16},
 6: {'empirical': 36, 'multi': 0, 'single': 36},
 7: {'empirical': 0, 'multi': 4, 'single': 4},
 8: {'empirical': 0, 'multi': 9, 'single': 9},
 9: {'empirical': 9, 'multi': 36, 'single': 9},
 10: {'empirical': 16, 'multi': 4, 'single': 4},
 11: {'empirical': 36, 'multi': 36, 'single': 0},
 12: {'empirical': 16, 'multi': 16, 'single': 0},
 13: {'empirical': 0, 'multi': 9, 'single': 9},
 14: {'empirical': 144, 'multi': 64, 'single': 16}}

In [119]:
sa = x_i2_sum / (t * n)

sb = 0
for key in ans_ab:
    sb += sum([x_it2[key][type] for type in types])
sb /= t
sb -= sa

sr = 0
for key in ans_ab:
    for i, type in enumerate(types):
        for b in types[i:]:
            sr += ans_all[type][b]
sr /= n
sr -= sa

empirical empirical
empirical multi
empirical single
multi multi
multi single
single single


In [97]:
# d_names = sorted(["1138-bus", "USpowerGrid", "qh882", "dwt-1005", "poli"])
# types = ["empirical", "multi", "single"]

# result = dict(
#     zip(
#         d_names,
#         [
#             dict(
#                 zip(
#                     types,
#                     [
#                         dict(zip(types, [0, 0, 0])),
#                         dict(zip(types, [0, 0, 0])),
#                         dict(zip(types, [0, 0, 0])),
#                     ],
#                 )
#             )
#             for _ in range(len(d_names))
#         ],
#     )
# )

# result["all"] = {
#     "empirical": {"empirical": 0, "multi": 0, "single": 0},
#     "multi": {"empirical": 0, "multi": 0, "single": 0},
#     "single": {"empirical": 0, "multi": 0, "single": 0},
# }
# score = dict(
#     zip(
#         d_names,
#         [
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#         ],
#     )
# )
# score_all = dict(zip(types, [0, 0, 0]))


# score_from_a = {
#     "Aが好ましい": 2,
#     "Aが少し好ましい": 1,
#     "変わらない": 0,
#     "Bが少し好ましい": -1,
#     "Bが好ましい": -2,
# }
# n = len(res) * 3
# for i, row in res.iterrows():
#     d = row.to_dict()
#     for uuid in d:
#         ans = d[uuid]
#         target_data = data[uuid]
#         a = target_data["A"]
#         b = target_data["B"]
#         dataset = target_data["dataset"]

#         score_a = score_from_a[ans]
#         score_b = -score_from_a[ans]
#         result[dataset][a][b] += score_a
#         result[dataset][b][a] += score_b
#         result["all"][a][b] += score_a
#         result["all"][b][a] += score_b
#         score[dataset][a] += score_a
#         score[dataset][b] += score_b
#         score_all[a] += score_a
#         score_all[b] += score_b

# for d_name in d_names:
#     for a in types:
#         for b in types:
#             result[d_name][a][b] /= n

# for a in types:
#     for b in types:
#         result["all"][a][b] /= n * 5